In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import interp
from itertools import cycle
from sklearn import svm, datasets, preprocessing
from sklearn.svm import SVC  
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, matthews_corrcoef, roc_curve, auc, make_scorer
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier

In [2]:
#read dataset
filename = 'C:\\AIA-2018\\examples\\feature-tables\\train-data_1526424532.csv'
data = pd.read_csv(filename)

#specify feature and target
Feature = data.drop(['class_id', 'Unnamed: 0'], axis=1)
Target  = data['class_id']

# shuffle the dataset
X = Feature.as_matrix()
y = Target.as_matrix()
X, y = shuffle(X, y, random_state=None)

#standardize data (mean = 0)
X = preprocessing.robust_scale(X)

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

In [ ]:
#Train algorithm
#svclassifier = SVC(C=10, class_weight={1: 10}, kernel='linear', probability=True, decision_function_shape='ovo')
svclassifier = SVC(C=0.1, cache_size=200, class_weight={1: 10}, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
svclassifier.fit(X_train, y_train)

In [3]:
# Grid Search 1
calculate_MCC = make_scorer(matthews_corrcoef)

parameters = [
  {'C': [0.1, 1, 10, 100], 'kernel': ['linear'], 'class_weight':['balanced',{1:8},{1:10},{1:12}]}
]

gridSVM = svm.SVC()
grid_search1 = GridSearchCV(gridSVM, parameters, scoring=calculate_MCC)
grid_search1.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'C': [0.1, 1, 10, 100], 'kernel': ['linear'], 'class_weight': ['balanced', {1: 8}, {1: 10}, {1: 12}]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(matthews_corrcoef), verbose=0)

In [ ]:
# Grid Search 2
calculate_MCC = make_scorer(matthews_corrcoef)

parameters = [
  {'C': [0.1, 1, 10, 100], 'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5], 'kernel': ['rbf','sigmoid'], 'class_weight':['balanced',{1:10},{1:20}]}
]

gridSVM = svm.SVC()
grid_search2 = GridSearchCV(gridSVM, parameters, scoring=calculate_MCC)
grid_search2.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'C': [0.1, 1, 10, 100], 'gamma': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.5], 'kernel': ['rbf', 'sigmoid'], 'class_weight': ['balanced', {1: 10}, {1: 20}]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(matthews_corrcoef), verbose=0)

In [ ]:
# Grid Search 3
calculate_MCC = make_scorer(matthews_corrcoef)

parameters = [
  {'C': [0.1, 1, 10, 100], 'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5], 'kernel': ['poly'], 'class_weight':['balanced',{1:10},{1:20}],'degree':[1,3,5], 'coef0':[1.0, 2.0]}
]

gridSVM = svm.SVC()
grid_search3 = GridSearchCV(gridSVM, parameters, scoring=calculate_MCC)
grid_search3.fit(X, y)

In [ ]:
import pandas as pd
data = pd.DataFrame(grid_search1.cv_results_)
grid_search1.best_params_

In [ ]:
# Runs the classifier and outputs the metrics
y_pred = svclassifier.predict(X_test)
print('Confusion matrix:')
print(confusion_matrix(y_test,y_pred,labels=[1,-1]),'\n')
print('Classification report:')
print(classification_report(y_test,y_pred,labels=[1,-1]),'\n')
print('Accuracy:')
print(accuracy_score(y_test, y_pred),'\n')
print('Matthew Correlation Coefficient:')
print(matthews_corrcoef(y_test, y_pred),'\n')
prob = svclassifier.predict_proba(X_test)

In [ ]:
# ROC Curve section
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, prob[:,1], pos_label=1, drop_intermediate=True)
roc_auc = auc(false_positive_rate, true_positive_rate)
plt.title('Receiver Operating Characteristic')
reg_per_img = (y_pred.size - 51) / 205
fp = false_positive_rate * reg_per_img
plt.plot(fp, true_positive_rate, 'b',
label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.xlim([-0,5])
plt.ylim([-0,1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive per Image')
plt.show()

In [ ]:
# Computations in order to properly scale the false positive ratio
unique, counts = np.unique(y, return_counts=True)
regions = y.shape[0]
neg_reg = counts[0]
pos_reg = counts[1]
num_img = 410
reg_per_img = regions / num_img
neg_reg_per_img = neg_reg / num_img
neg_reg_per_img
scaling = 5 / neg_reg_per_img

In [ ]:
# Receiver Operating Characteristic (ROC) with cross validation
cv = StratifiedKFold(n_splits=6)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

i = 0
matthew = 0
for train, test in cv.split(X, y):
    probas_ = svclassifier.fit(X[train], y[train]).predict_proba(X[test])
    y_pred = svclassifier.predict(X[test])
    matthew = matthew + matthews_corrcoef(y[test], y_pred)
    
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    
    #fpr = neg_reg_per_img * fpr
    
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    i += 1
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Luck', alpha=.8)

#mean_fpr = mean_fpr * neg_reg_per_img

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()
print(matthew/6)

In [ ]:
SVM_classif = SVC(C=10, class_weight='balanced', kernel='linear', probability=True)
clf = AdaBoostClassifier(SVM_classif,n_estimators=50,learning_rate=1, algorithm='SAMME.R')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Confusion matrix:')
print(confusion_matrix(y_test,y_pred,labels=[1,-1]),'\n')
print('Classification report:')
print(classification_report(y_test,y_pred,labels=[1,-1]),'\n')
print('Accuracy:')
print(accuracy_score(y_test, y_pred),'\n')
print('Matthew Correlation Coefficient:')
print(matthews_corrcoef(y_test, y_pred),'\n')
prob = clf.predict_proba(X_test)